In [1]:
#===============
#   IMPORTS
#===============

# Fundamentals
import numpy as np
import pandas as pd

# Dash
import dash
from dash import html
from dash import dcc
import dash_bootstrap_components as dbc
from dash.dependencies import Output, Input

# Dash in JupyterLab
from jupyter_dash import JupyterDash

# Plotly
import plotly.graph_objects as go
import plotly.io as pio
from plotly.subplots import make_subplots

from plotly.tools import mpl_to_plotly

pio.renderers.default = 'iframe'


#-STYLES----------------------------------
linestyles = ['-', '--', '-.', ':', (0, (5, 2, 1, 2, 1, 2))]
markers = ['o', 'x', 's', '^', 'd']
colors = ['#0077BB', '#33BBEE', '#009988', '#EE7733', '#CC3311', '#EE3377', '#BBBBBB']
colors4 = ['#0077BB', '#EE7733', '#CC3311', '#EE3377']
blues = ['#eff3ff','#c6dbef','#9ecae1','#6baed6','#3182bd','#08519c']
greys5 = ['#f7f7f7','#cccccc','#969696','#636363','#252525']
greys6 = ['#f7f7f7','#d9d9d9','#bdbdbd','#969696','#636363','#252525']
greys7 = ['#f7f7f7','#d9d9d9','#bdbdbd','#969696','#737373','#525252','#252525']
greys8 = ['#ffffff','#f0f0f0','#d9d9d9','#bdbdbd','#969696','#737373','#525252','#252525']
greys9 = ['#ffffff','#f0f0f0','#d9d9d9','#bdbdbd','#969696','#737373','#525252','#252525','#000000']
hatches = ['/', '\\', '|', '-', '+', 'x', 'o', 'O', '.', '*']


#====================
#   PY INCLUSIONS
#====================

# Data import
import baptistery_data_input as bdi
prism_data, levelling_data, extensimeter_data = bdi.readAllData()
prism_pos, levelling_pos, extensimeter_pos, positions = bdi.readSensorPositions()

def reformatPlot(fig, size=None, secondary=False):
    if size != None:
        fig.update_layout(height=size[1], width=size[0])
        
    fig.update_layout(xaxis_showline=True,
                      xaxis_mirror='ticks',
                      xaxis_linewidth=2,
                     yaxis_showline=True,
                     yaxis_mirror='ticks',
                     yaxis_linewidth=2)
    fig.update_layout(
        xaxis_ticks="inside",
        xaxis_ticklen=8,
        xaxis_tickwidth=2,
        yaxis_ticks="inside",
        yaxis_ticklen=8,
        yaxis_tickwidth=2
    )
    fig.update_layout(
        font_family="Roboto",
        hoverlabel_font_family="Roboto",
        xaxis_title_font_size=16,
        xaxis_tickfont_size=16,
        xaxis_showgrid=True,
        yaxis_title_font_size=16,
        yaxis_tickfont_size=16,
        yaxis_showgrid=True,
    )
    
    if secondary:    
        fig.update_layout(
                yaxis2=dict(
                title_font_size=16,
                tickfont_size=16,
                showgrid=False,
                showline=True,
                linewidth=2,
                ticks='inside',
                ticklen=8,
                tickwidth=2,
                title_font_color='gray',
                tickcolor='lightgray',
                tickfont_color='gray'
            ),
            yaxis_mirror=False
        )

    return fig

def selectPrismSection(n):
    """
    NOTE: for now, the function excludes prisms in the 
    1xx series.    
    """
    full_section = {
        '01':['01','07'],
        '02':['02','08'],
        '03':['03','09'],
        '04':['04','10'],
        '05':['05','11'],
        '06':['06','12'],
        '07':['07','01'],
        '08':['08','02'],
        '09':['09','03'],
        '10':['10','04'],
        '11':['11','05'],
        '12':['12','06']
    }
    n = full_section[n]
   
    selected_sensors = [p for p in prism_pos.index if p.endswith(n[0]) and not (p.startswith('1'))]
    selected_sensors += [p for p in prism_pos.index if p.endswith(n[1]) and not (p.startswith('1'))]
    
    return selected_sensors    


def rotTraslPrism(df, date):
    """
    Returns east and z coordinates of prisms
    in the DataFrame *df* for the date *date*.
    """
    x = df.loc[date, (slice(None), 'x')].values
    y = df.loc[date, (slice(None), 'y')].values
    z = df.loc[date, (slice(None), 'z')].values
    
    # Traslation
    ref_x, ref_y = 15.184322095298622, -0.01676310147012092
    x1 = x - ref_x
    y1 = y - ref_y
    
    # Rotation
    a = -np.arctan(y1/x1)
    east = x1*np.cos(a) - y1*np.sin(a)
    north = x1*np.sin(a) + y1*np.cos(a)

    if (north > 10**(-6)).any():
        print("ERROR: north coordinate is not zero")
        return 1
    
    return east, z

def xyToEN(x, y):
    """
    Converts (x, y) coordinates of prisms to (E, N) ones.
    It takes either single or array-like coordinates.
    """
    ref_x, ref_y = 15.184322095298622, -0.01676310147012092
    rot = np.deg2rad(37.1)
    e = x - ref_x
    n = y - ref_y
    eR = e*np.cos(rot) - n*np.sin(rot)
    nR = e*np.sin(rot) + n*np.cos(rot)
   
    return eR, nR

In [30]:
def figureLevellingChecks():
    """
    Returns the contents of a div (a list), with plots of 
    levelling data vs prism data (vertical component);
    each plot is contained in a separate Graph object. 
    """
    levelling_pruned = levelling_data[[c for c in levelling_data.columns if c[0] == 'L']]
    corresponding_prisms = [str(p) for p in [205, 305, 206, 306, 207, 307, 208, 308, 209, 309, 210, 310, 211, 311, 212, 312, 201, 301, 202, 302, 203, 303, 204, 304]]
    prism_z_rel = (x:= prism_data.loc[:, (corresponding_prisms, 'z')]) - x.iloc[0]
    
    def singleFigure(l,p,lname,pname):
        """
        Actually makes the plot. l and p must be array-like or Series
        with relative vertical displacements of levelling and prism,
        respectively.
        """
        fig = go.Figure(layout_template=None)
        fig.update_layout(margin = dict(t=40, b=40))
        fig = make_subplots(specs=[[{"secondary_y": True}]],
                            figure=fig)
        fig = reformatPlot(fig, size=[1200, 350], secondary=True)

        fig.add_trace(
            go.Scatter(
                x=levelling_data.index,
                y=l,
                mode='markers+lines',
                name=lname,
                marker_color='#999933',
                line_color='#999933'
            ),
            secondary_y=False,
        )  
        
        fig.add_trace(
            go.Scatter(
                x=prism_data.index,
                y=p,
                mode='markers+lines',
                name=pname,
                marker_color='#882255',
                line_color='#882255'
            ),
            secondary_y=False,
        )  

        fig.add_trace(
            go.Scatter(
                x=extensimeter_data.index,
                y=extensimeter_data['F4F8', 'temp'].rolling(24).mean(),
                line_dash='dot',
                line_color='gray',
                name='Temperature'
            ),
            secondary_y = True,
        )    

        fig.add_annotation(
            xref="x domain",
            yref="y domain",
            x=0.02,
            y=0.95,
            text='<b>'+ lname + '-' + pname +'</b>',
            font_family='Roboto',
            font_color='black',
            font_size=14,
            borderpad=4,
            bordercolor='black',
            borderwidth=1.5,
            showarrow=False
        )

        fig.update_yaxes(title_text="Displacement [mm]", secondary_y=False)
        fig.update_yaxes(title_text="Temperature [°C]", secondary_y=True)

       # fig.update_layout(
       #     legend=dict(
       #         x=0.70, y=0.95
       #     )
       # )

        return fig
    
    children = [] 
    for lname, pname in zip(levelling_pruned.columns, corresponding_prisms):
        graph = dcc.Graph(figure=singleFigure(levelling_pruned[lname], prism_z_rel[pname], lname, pname))        
        children.append(graph)
    
    return children

In [34]:
levelling_data

,RIF1,L1,L2,L3,L4,L5,L6,L7,L8,L9,...,L16,L17,L18,L19,L20,L21,L22,L23,L24,Caposaldo_IGM
date,,,,,,,,,,,,,,,,,,,,,
2013-12-17,0,0.000000,0.00000,0.00000,0.000000,0.00000,0.000000,0.00000,0.000000,0.0000,...,0.00000,0.000000,0.000000,0.00000,0.00000,0.000000,0.000000,0.000000,0.0000,0.000000
2014-03-07,0,0.122449,-0.04898,-0.02449,-0.138776,-0.02449,-0.097959,0.02449,-0.081633,0.0000,...,0.02449,0.065306,-0.016327,0.04898,0.04898,0.057143,-0.008163,0.073469,0.0000,-0.073469
2014-04-08,0,0.212449,0.07102,0.07551,-0.068776,0.01551,-0.027959,0.06449,-0.061633,0.0300,...,-0.07551,0.025306,-0.046327,0.00898,-0.04102,0.197143,0.081837,0.173469,0.0800,-0.003469
2014-05-08,0,0.202449,0.09102,0.00551,-0.118776,0.15551,-0.047959,0.16449,0.058367,0.1400,...,0.02449,0.115306,0.053673,0.12898,0.03898,0.167143,0.061837,0.163469,0.0900,-0.013469
2014-06-09,0,0.302449,0.10102,0.16551,-0.018776,0.16551,0.042041,0.21449,0.048367,0.2000,...,0.01449,0.145306,0.053673,0.15898,0.06898,0.277143,0.161837,0.273469,0.1400,-0.193469
2014-07-01,0,0.342449,0.19102,0.14551,0.001224,0.01551,0.042041,0.03449,-0.101633,0.0400,...,-0.05551,0.025306,-0.036327,0.06898,-0.06102,0.297143,0.181837,0.303469,0.1800,-0.273469
2014-08-04,0,0.132449,0.06102,0.09551,-0.108776,0.15551,-0.007959,0.17449,0.008367,0.1200,...,-0.00551,0.105306,-0.006327,0.13898,0.03898,0.197143,0.061837,0.193469,0.0800,-0.313469
2014-09-03,0,0.172449,0.12102,-0.03449,-0.138776,0.10551,-0.037959,0.07449,-0.041633,0.0000,...,0.02449,0.105306,0.033673,0.19898,0.10898,0.237143,0.151837,0.233469,0.1500,-0.243469
2014-10-22,0,0.132449,0.17102,0.16551,0.111224,0.23551,-0.007959,0.17449,0.118367,0.1900,...,-0.06551,0.045306,0.013673,0.19898,0.01898,0.237143,0.051837,0.203469,0.1600,-0.183469


In [59]:
levelling_pruned = levelling_data[[c for c in levelling_data.columns if c[0] == 'L']]
corresponding_prisms = [str(p) for p in [205, 305, 206, 306, 207, 307, 208, 308, 209, 309, 210, 310, 211, 311, 212, 312, 201, 301, 202, 302, 203, 303, 204, 304]]
prism_z_rel =  ((x:= prism_data.loc[:, (corresponding_prisms, 'z')]) - x.iloc[0])*1000.

def singleFigure(l,p,lname,pname):
    """
    Actually makes the plot. l and p must be array-like or Series
    with relative vertical displacements of levelling and prism,
    respectively.
    """
#
    fig = go.Figure(layout_template=None)
    fig.update_layout(margin = dict(t=40, b=40))
    fig = make_subplots(specs=[[{"secondary_y": True}]],
                        figure=fig)
    fig = reformatPlot(fig, size=[1200, 350], secondary=True)

    fig.add_trace(
        go.Scatter(
            x=levelling_data.index,
            y=l,
            mode='markers+lines',
            name=lname,
            marker_color='#999933',
            line_color='#999933'
        ),
        secondary_y=False,
    )  

    fig.add_trace(
        go.Scatter(
            x=prism_data.index,
            y=p[:,0],
            mode='markers+lines',
            name=pname,
            marker_color='#882255',
            line_color='#882255'
        ),
        secondary_y=False,
    )  

    fig.add_trace(
        go.Scatter(
            x=extensimeter_data.index,
            y=extensimeter_data['F4F8', 'temp'].rolling(24).mean(),
            line_dash='dot',
            line_color='gray',
            name='Temperature'
        ),
        secondary_y = True,
    )    

    fig.add_annotation(
        xref="x domain",
        yref="y domain",
        x=0.02,
        y=0.95,
        text='<b>'+ lname + '-' + pname +'</b>',
        font_family='Roboto',
        font_color='black',
        font_size=14,
        borderpad=4,
        bordercolor='black',
        borderwidth=1.5,
        showarrow=False
    )

    fig.update_yaxes(title_text="Displacement [mm]", secondary_y=False)
    fig.update_yaxes(title_text="Temperature [°C]", secondary_y=True)

   # fig.update_layout(
   #     legend=dict(
   #         x=0.70, y=0.95
   #     )
   # )

    return fig

In [60]:
prism_z_rel[pname].values[:,0]

array([0.  , 0.  , 0.1 , 0.4 , 0.8 , 0.8 , 0.8 , 0.6 , 0.7 , 0.4 , 0.4 ,
       0.4 , 0.4 , 0.5 , 0.7 , 0.7 , 0.9 , 0.8 , 0.6 , 0.3 , 0.1 , 0.1 ,
       0.16, 0.27, 0.47, 0.33, 0.27])

In [63]:
levelling_pos

,angle,radius,z,type
RIF1,9.70,9.40,1.6,level
L1,197.38,9.95,1.6,level
L2,194.20,15.25,1.6,level
L3,160.75,9.95,1.6,level
L4,163.87,15.25,1.6,level
L5,130.24,9.95,1.6,level
L6,133.64,15.25,1.6,level
L7,100.44,9.95,1.6,level
L8,103.34,15.25,1.6,level
L9,74.37,9.95,1.6,level


In [61]:
lname = 'L11'
pname = '210'
figu = singleFigure(levelling_pruned[lname].values , prism_z_rel[pname].values, lname, pname)
figu.show()


In [38]:
prism_z_rel[pname]

,z
date,
2013-12-17,0.00
2014-01-30,0.00
2014-03-07,0.10
2014-04-11,0.40
2014-06-09,0.80
2014-07-01,0.80
2014-08-05,0.80
2014-09-17,0.60
2014-10-22,0.70
